In [ ]:
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
import os
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
# response = requests.get('https://www.merit-times.com.tw/NewsPage.aspx?unid=903451')
response = requests.get('https://rate.bot.com.tw/xrt?Lang=zh-TW')
soup = BeautifulSoup(response.text,"html.parser")
body_content = soup.body
for script_or_style in body_content(['script','style']):
    script_or_style.extract()

table_content = body_content.find(title='牌告匯率')
table_lines = table_content.get_text(separator='\n')
cleaned_content = "\n".join(
    [line.strip() for line in table_lines.splitlines() if line.strip()]
    )


model = genai.GenerativeModel(
    "gemini-2.0-flash-exp",
    system_instruction='''
    你的任務是取出指定的內容,並輸出文字
    請依照下面的指示:
    樣本:
    ```
    澳幣 (AUD)
    澳幣 (AUD)
    20.15
    20.93
    20.365
    20.71
    ```
    輸出的格式:
    澳幣(AUD):
    現金匯率(本行買入):20.15
    現金匯率(本行賣出):20.93
    即期匯率(本行買入):20.365
    即期匯率(本行賣出):20.365 
    ==================   
    '''
)

response = model.generate_content(cleaned_content)
result_text = response.text
print(result_text)




美金(USD):
現金匯率(本行買入):32.28
現金匯率(本行賣出):32.95
即期匯率(本行買入):32.605
即期匯率(本行賣出):32.755
港幣(HKD):
現金匯率(本行買入):4.04
現金匯率(本行賣出):4.244
即期匯率(本行買入):4.161
即期匯率(本行賣出):4.231
英鎊(GBP):
現金匯率(本行買入):39.5
現金匯率(本行賣出):41.62
即期匯率(本行買入):40.395
即期匯率(本行賣出):41.025
澳幣(AUD):
現金匯率(本行買入):20.06
現金匯率(本行賣出):20.84
即期匯率(本行買入):20.275
即期匯率(本行賣出):20.62
加拿大幣(CAD):
現金匯率(本行買入):22.22
現金匯率(本行賣出):23.13
即期匯率(本行買入):22.55
即期匯率(本行賣出):22.88
新加坡幣(SGD):
現金匯率(本行買入):23.6
現金匯率(本行賣出):24.51
即期匯率(本行買入):24.07
即期匯率(本行賣出):24.29
瑞士法郎(CHF):
現金匯率(本行買入):35.27
現金匯率(本行賣出):36.47
即期匯率(本行買入):35.88
即期匯率(本行賣出):36.27
日圓(JPY):
現金匯率(本行買入):0.2011
現金匯率(本行賣出):0.2139
即期匯率(本行買入):0.2079
即期匯率(本行賣出):0.2129
南非幣(ZAR):
現金匯率(本行買入):-
現金匯率(本行賣出):-
即期匯率(本行買入):1.706
即期匯率(本行賣出):1.796
瑞典幣(SEK):
現金匯率(本行買入):-
現金匯率(本行賣出):-
即期匯率(本行買入):2.92
即期匯率(本行賣出):3.04
紐元(NZD):
現金匯率(本行買入):18.03
現金匯率(本行賣出):18.88
即期匯率(本行買入):18.36
即期匯率(本行賣出):18.66
泰幣(THB):
現金匯率(本行買入):0.8356
現金匯率(本行賣出):1.0256
即期匯率(本行買入):0.946
即期匯率(本行賣出):0.992
菲國比索(PHP):
現金匯率(本行買入):0.4931
現金匯率(本行賣出):0.6251
即期匯率(本行買入):-
即期匯率(本行賣出):-
印尼幣(I

In [28]:
import google.generativeai as genai
import typing_extensions as typing

class Rate(typing.TypedDict):
    buy:float | None
    sell:float | None

class Recipe(typing.TypedDict):
    幣別:str
    類別1:Rate
    類別2:Rate

genai.configure(api_key=os.environ['GEMINI_API_KEY'])

model1 = genai.GenerativeModel(
    "gemini-2.0-flash-exp",
    system_instruction='''    
    1.如果無法轉成float,請使用None替代
    **輸入的格式:**
    ```
    澳幣(AUD):
    現金匯率(本行買入):20.15
    現金匯率(本行賣出):20.93
    即期匯率(本行買入):20.365
    即期匯率(本行賣出):20.71 
    ```

    **轉換輸入的格式成為輸出json schema:**
    ```
    [
        {
           '幣別':'澳幣',
           '類別1':{
                    'buy':20.15,
                    'sell':20.93
                }
           '類別2':{
                    'buy':20.365,
                    'sell':20.71
                } 
        }
    ]
    ```
    '''
)
response =model.generate_content(
    result_text,
    generation_config = genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema=list[Recipe]
    ) 
)
print(response.text)


[
  {
    "幣別": "美金(USD)",
    "類別1": {
      "buy": 32.28,
      "sell": 32.95
    },
    "類別2": {
      "buy": 32.605,
      "sell": 32.755
    }
  },
  {
    "幣別": "港幣(HKD)",
    "類別1": {
      "buy": 4.042,
      "sell": 4.246
    },
    "類別2": {
      "buy": 4.163,
      "sell": 4.233
    }
  },
  {
    "幣別": "英鎊(GBP)",
    "類別1": {
      "buy": 39.66,
      "sell": 41.78
    },
    "類別2": {
      "buy": 40.555,
      "sell": 41.185
    }
  },
  {
    "幣別": "澳幣(AUD)",
    "類別1": {
      "buy": 20.21,
      "sell": 20.99
    },
    "類別2": {
      "buy": 20.425,
      "sell": 20.77
    }
  },
  {
    "幣別": "加拿大幣(CAD)",
    "類別1": {
      "buy": 22.27,
      "sell": 23.18
    },
    "類別2": {
      "buy": 22.6,
      "sell": 22.93
    }
  },
  {
    "幣別": "新加坡幣(SGD)",
    "類別1": {
      "buy": 23.75,
      "sell": 24.66
    },
    "類別2": {
      "buy": 24.22,
      "sell": 24.44
    }
  },
  {
    "幣別": "瑞士法郎(CHF)",
    "類別1": {
      "buy": 35.56,
      "sell": 36.76
    },
    "類別2"